# Introduction

This first notebook functions as a general introduction to the Python tools that accompany the book. The notebook showcases all the software components and introduces the syntax necessary to use them. The entire tutorial is written in Python, and the reader is expected to have a passing knowledge of the language. Python is an easy language to pick up quickly; a good introduction can be found [here](https://docs.python.org/3/tutorial/index.html).

The tutorial is written using the [Jupyter Notebook](http://jupyter.org/) webapp which allows the integration of sections of text like this one with code blocks like the following:

In [ ]:
1+1

Code blocks (cells in the notebook terminology) can be run after being selected using the 'run cell' button in the toolbar above or the keyboard shortcut Ctrl-Enter. The output of the code (the value of the last evaluated expression and any kind of printing produced with side-effecting functions) is shown below the code cell. 

Every cell (code and text) can be edited and the modifications can be saved with the save button in the toolbar.

# The theorem prover

The core of the software is the theorem prover that implements the logical calculus we introduce in the book. Let's start by importing everything that's in the theorem prover module (again to run the following piece of code use the 'run cell' button or press Ctrl-Enter after selecting it):

In [ ]:
from mb.tp import *

The package ``mb`` contains all the modules of the tutorial, and the module ``tp`` is the one containing the theorem prover. We start by defining some formulae. There are five types of formulae we can define:
1. atoms
2. variables
3. implications
4. tensor products
5. monadic formulae

Each formula has a type, but it's only necessary to directly specify the type of atoms and variables as the constructors for the other three types of formulae are smart enough to figure out their type from the type of their constituent parts. Let's define a couple of atoms (don't forget to run the cell):

In [ ]:
a = Atom('a', AtomicType('t'))
b = Atom('b', AtomicType('s'))

We have defined two atoms ``a`` and ``b`` using the constructor ``Atom``. The constructor takes two arguments:
1. an identifier, which can be anything but it's usually a string that gives a name to the atom
2. a type, which is usually an atomic type

In this case both atoms have as type an atomic type, which is created with the constructor ``AtomicType`` which takes as its only argument an identifier that determines the type (again it could be anything but a string makes it clearer what we are doing). Notice that we don't have to create a new type object for each atom we create. We can save typing (and a little bit of memory) by creating a single type and reusing it multiple times:

In [ ]:
type_z = AtomicType('z')
c = Atom('c', type_z)
d = Atom('d', type_z)

Let's try to prove something with our freshly created atoms. We start with the simple sequent $a \vdash a$. Sequents are constructed with the constructor ``Sequent``. ``Sequent`` takes two arguments: a list of formulae representing the left-hand side of the turnstile and a single formula (we are working with linear logic) that represents the right-hand side of the turnstile.

In [ ]:
s = Sequent([a],a)

To prove the sequent we use the function ``prove``. The function takes a single sequent as its argument and returns a ``ProofResults`` object, a handy wrapper object with a number of methods that makes the operation of analyzing the single proofs much easier.

In [ ]:
proofs = prove(s)

To visualize the proofs we can just invoke the ``show`` method on the object returned by ``prove``:

In [ ]:
proofs.show()

Not a particularly exciting proof but still a proof. We can try and construct more complex sequent like $\lozenge a, \lozenge a \multimap b \vdash \lozenge b$.

In [ ]:
diamond_a = Monad(a,'')
implication = Implication(diamond_a, b)
diamond_b = Monad(b,'')
more_interesting_sequent = Sequent([diamond_a, implication], diamond_b)

As you can see, we use two new constructors here, ``Monad`` and ``Implication``. The first one is used to construct monadic resources. It takes two arguments: the formula over which the monadic connective has scope and the modality of the monad (it can be anything, a string makes the visualization more clear). Here we choose an anonymous modality that we represent with the empty string ``''``. ``Implication`` is used to construct linear implications. Its two arguments represent respectively the left-hand and the right-hand side of the implication. Let's ask the prover to prove the sequent for us and to visualize the resulting proofs.

In [ ]:
more_interesting_proofs = prove(more_interesting_sequent)
more_interesting_proofs.show()

A couple of comments. First of all, you may have noticed that some formulae in the proofs display are red. Those are the formulae on which the rules indicated in each step are operating (if we read the proof trees in a bottom-up fashion, which is the same way in which they are constructed). Then notice that we get quite a few proofs. We can count them or ask the ``ProofResults`` object for their number:

In [ ]:
len(more_interesting_proofs)

We get six proofs, but if we had a way to reduce them to some kind of normal form we would expect only two proofs. This is exactly what we are going to do in the next section using the Curry-Howard correspondence between proofs and lambda terms. But before introducing the correspondence we will look at a couple more basic tools of the prover.

First of all we still have to introduce two more constructors used to define variable resources and resources coupled with the multiplicative tensor $\otimes$. For the first kind of resource we use constructor ``Variable`` which works exactly in the same way as ``Atom``:

In [ ]:
x = Variable('x', type_z)

For constructing tensor resources we have (unsurprisingly) a ``Tensor`` constructor which works as the reader may expect:

In [ ]:
c_times_d = Tensor(c,d)
c_times_d.show()

Notice that we can call the ``show`` method also on formulae and sequents in order to visualize them. Now we can try to prove the following sequent $c \otimes d, x \multimap d \multimap a \vdash a$. What we expect is the tensor to be broken apart and the $c$ resource to be unified with the variable $x$. To construct the double implication we can use a shortcut: we can use the exponentiation ``**`` symbol to indicate that the "base" implies the "exponent":

In [ ]:
long_implication = x ** d ** a
long_implication.show()

This saves us some typing. We selected ``**`` because it vaguely resembles $\multimap$ and because it is right associative like $\multimap$ is.

Similarly for tensor we can the multiplication operator ``*``:

In [ ]:
c_times_d = c * d
c_times_d.show()

And now the sequent:

In [ ]:
tensor_sequent = Sequent([c_times_d, long_implication],a)
tensor_proofs = prove(tensor_sequent)
tensor_proofs.show()

### Exercises

Construct and prove the following sequents:
1. $a \multimap b, \lozenge a \vdash \lozenge b $
2. $\lozenge \lozenge a \vdash \lozenge a$

In [ ]:
# Type the solution to the exercises here

# Curry-Howard Correspondence

We are going to use the Curry-Howard correspondence for two main purposes: 1. as already stated, as a tool to distinguish and group sequent proofs, 2. as a way to identify different meanings associated with the proofs via their proof terms.

Using the correspondence in the theorem prover is quite straighforward, we call the ``group_proofs`` method on the object returned by ``prove`` and the prover automatically creates proof terms for us and groups the proofs together. Let's try with a sequent representing a well known case of multiple readings due to quantifier scope ambiguities, something like *every captain hates a whale*. We start by constructing the resources:

In [ ]:
e_type = AtomicType('e')
t_type = AtomicType('t')
e_t_type = ArrowType(e_type, t_type) # Here we construct the e->t directly to simplify the other definitions

goal = Atom('h', t_type)
captain = Atom('c', e_t_type)
whale = Atom('w', e_t_type)
c = Atom('c', e_type)
w = Atom('w', e_type)
hates = w ** c ** goal
x = Variable('x', t_type)
y = Variable('y', t_type)
every = captain ** (c ** x) ** x
a = whale ** (w ** y) ** y

achab_and_moby = Sequent([every, captain, hates, a, whale], goal)
proofs = prove(achab_and_moby)

If we now ask the prover how many proofs there are for the sequent ``achab_and_moby`` we don't exactly get what we expect:

In [ ]:
len(proofs)

This is because the prover is not particularly clever in the construction of the proofs and considers proofs that are simple permutations of the applications of the same rules as different proofs. We can however ask it to group them:

In [ ]:
proofs.group_proofs()
len(proofs)

And we now get the number we expect from what we have learned in our first class in generalized quantifiers. We can also ask the prover to show us a witness proof for each group, by passing to the method ``show`` an additional argument ``show_only_witness`` and setting it to ``True``:

In [ ]:
proofs.show(True)

The two proof terms associated with the right-hand side of the sequent at the root of the trees represent the two different readings, but clearly they are not easy to read, given the anonymity of the constants chosen for us by the prover. However we can pass to the ``prove`` function a list of lambda terms to be associated with the resources in the left-hand side of the sequent. These terms are going to be used to construct the proof terms that encode the reading associated with each proof. The module ``mb.lambda_calc`` contains all the necessary machinery to define lambda terms. In our case we will only use the ``Const`` constructor to create constants:

In [ ]:
from mb.lambda_calc import Const
proofs = prove(achab_and_moby,[Const('forall'), Const('captain'), Const('hate'), Const('exist'), Const('whale')])
proofs.group_proofs()
proofs.show(True)

Of course we can also construct other types of lambda terms besides constants:
- variables: ``Var('x')``
- abstractions: ``Abstraction(Var('x'), Const('body'))``, or more concisely ``Var('x') ^ Const('body')``
- applications: ``Application(Const('f'), Const('a'))``, or more concisely ``Const('f')(Const('a'))``
- pairs: ``Pair(Const('a'), Const('b'))``, or more concisely ``Const('a') * Const('b')``
- projections: ``FirstProjection(Pair(Const('a'), Const('b')))`` and ``SecondProjection(Pair(Const('a'), Const('b')))``
- monadic units: ``Unit(Const('a'))``
- and finally bind: ``Bind(Unit(Const('a')), Var('x') ^ (Unit(Const('f')(Var('x')))))``, or more concisely ``Unit(Const('a')) ** (Var('x') ^ (Unit(Const('f')(Var('x')))))``

Let's see all this in code:

In [ ]:
from mb.lambda_calc import Var, Abstraction, Application, Pair, FirstProjection, SecondProjection, Unit, Bind
(Var('x')).show()
(Var('x') ^ Const('body')).show()
Const('f')(Const('a')).show()
(Const('a') * Const('b')).show()
FirstProjection(Pair(Const('a'), Const('b'))).show()
Unit(Const('a')).show()
(Unit(Const('a')) ** (Var('x') ^ (Unit(Const('f')(Var('x')))))).show()

# From Words to Meanings
So far there has been a little bit of magic involved in the construction of the basic sequents corresponding to linguistic expressions. In this section we will explore the construction of a proper lexicon and an associated grammar to be used with a parser that's going to feed its results to the theorem prover automatically.
We are going to use the categorial grammar presented in the book. The main advantage of this grammar is that it corresponds in a fairly direct way to the linear logic system we have played with so far. We are basically just splitting implication into two directional kinds of implications, the forward and backward slash. We also need to switch from using unordered bags of semantic resources (to represent the semantic material of a sentence) to linearly ordered sequences of semantic resources, reflecting the order of the elements of the linguistic expressions we will model. We start by importing the module for the categorial grammar ``mb.cg``:

In [ ]:
from mb.cg import *

Given that the linear logic calculus and the categorial grammar are so similar, we mostly used the same names to refer to the same kind of objects. This means that the import we just did has shadowed most of the imports from the module ``mb.tp``. So now if we use the constructor ``Atom`` we are constructing an atom of the grammar implemented in the ``mb.cg`` module rather than an atom of the ``mb.tp`` module. If (for some reason) we wanted to use both modules at the same time we should import their objects in a qualified manner (for instance by using ``import mb.tp``).

If you are familiar with categorial grammars, you know that the objects we deal with are formulae that represent different kinds of grammatical objects, constructed starting from atomic objects (e.g. ``np`` and ``s``, representing respectively noun phrases and sentences), and combined with different kinds of binary or unary connectives, such as the two directional implications ``\`` and ``/``, possibly tensors, and in our case also monadic unary operators, similar to those used in the linear logic calculus. Let's start and create some of these objects:

In [ ]:
np = Atom('np', AtomicType('e'))
s = Atom('s', AtomicType('t'))

Here we have created two atomic formulae that we are going to use to represent respectively noun phrases and sentences. Notice that our formulae are typed, exactly as was the case for the logical calculus. This is because we also have the possibility to use variables together with constant formulae, for instance to express the syntactic type of a word like *and* that works across different kinds of syntactic expressions (*Ash and Gianluca*, *dogs and cats*, *Ash wrote a book and Gianluca corrected it*).

Now let's create some implications. We are going to write implication with the argument always on the side from which it is expected and always "under" the slash, i.e. we write ``a / b`` meaning that we expect an argument of kind ``b`` on the right and we produce a compound expression of type ``a``, and *vice versa* ``b \ a`` for an expression expecting another expression of kind ``b`` on the left to produce an ``a``. In the Python code the rightward looking implication is constructed with the ``RightImplication`` constructor, and the leftward looking one with ``LeftImplication``. To reduce keystrokes (and to make things easier to read), we can instead write ``a << b`` for the rightward implication (the object expects a ``b`` on its right to produce an ``a``), and similarly ``b >> a`` for the leftward implication. For instance we can create the type of intransitive verbs ``np \ s`` as follows (as usual we can use the ``show`` method to get a readable representation of all the objects we play with):

In [ ]:
iv = np >> s
iv.show()

And now we are ready to produce our first derivation: $np , np \backslash s \vdash s$, or in code

In [ ]:
sequent = Sequent([np, iv], s)

To prove the sequent and see the resulting proofs we use the same commands we used for the logical calculus:

In [ ]:
proofs = prove(sequent)
proofs.show()

Of course we can also create tensors:

In [ ]:
(np * np).show()

And monads:

In [ ]:
Monad(s, 'ci').show()

This is not particularly exciting. The real use of a grammar is to describe the process that gets us from linguistic expressions to their syntactic structure and ultimately their meaning. For this we need a lexicon, which we construct in our implementation with ``Lexicon``:

In [ ]:
lexicon = Lexicon()

Now we have an empty lexicon. We can start to fill it up with the method ``add_entry``. This method expects a "word" (which can be whatever you want in principle but should really be a string **without spaces**, more on this below)), a syntactic category expressed as a formula of the kind just discussed, and a meaning object in the form of an appropriate lambda term. So let's add some simple entries to our lexicon

In [ ]:
lexicon.add_entry('Ash', np, Const('ash'))
lexicon.add_entry('snores', iv, Const('snore'))

Now that we have a very minimal lexicon we can introduce the magical function ``prove_sentence`` that, given a sentence, a target syntactic category and a lexicon, returns us all the reading of said sentence! (as usual we pass ``True`` to the ``show`` method to get back the decorated proofs)

In [ ]:
proofs = prove_sentence('Ash snores', s, lexicon)
proofs.show(True)

Of course we can spice things up a bit and play with transitive verbs:

In [ ]:
tv = iv << np
lexicon.add_entry('envies', tv, Const('envy'))
lexicon.add_entry('Gianluca', np, Const('gianluca'))
proofs = prove_sentence('Ash envies Gianluca', s, lexicon)
proofs.show(True)

The function ``prove_sentence`` is actually doing pretty basic things: first of all it tokenizes in the simplest possible way the sentence by simply splitting it up at every space (that's why you can't put spaces in the word part of a lexical entry unless you want to have problems later on), then it looks up every token in the lexicon and composes for us the sequent to prove, while associating to it all the necessary meaning bindings.

Now as an exercise try to test the limits (there are many) of this little grammatical system. Try to play with quantifiers and similar objects and see what you can get out of it.

In [ ]:
# Have fun with the grammar!